In [64]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from numpy import exp
import math


#TASK 1
#fruit = pd.read_csv("H:/Machine Learning/fruit_data_with_colors.txt", sep='\t')
fruit = pd.read_csv("C:/Users/Joshua/Desktop/University/Machine Learning/fruit_data_with_colors.txt", sep='\t')
fruit.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [65]:
#TASK 2
fruitVector = fruit[["mass","width","height","color_score","fruit_name"]]

In [66]:
X = fruitVector.iloc[:,:-1].values
y = fruitVector.iloc[:,4].values

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 4321)

In [68]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train[0:4])

[[-1.82753952 -1.56031839 -2.94486615  0.54956872]
 [ 0.89715576  0.36161885  2.22491959 -0.68696091]
 [-1.01013093 -1.42303716 -0.27658964 -0.68696091]
 [ 0.07974718  0.22433762 -0.69350784  0.41217654]]


In [69]:
for x in range(1,9):
    classifier = KNeighborsClassifier(n_neighbors=x)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print("kNN %):", metrics.accuracy_score(y_test, y_pred)*100)

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
kNN %): 100.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
kNN %): 100.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
kNN %): 100.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
kNN %): 100.0
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 0 0 6]]
kNN %): 66.66666666666666
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 1 0 5]]
kNN %): 61.111111111111114
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 0 0 6]]
kNN %): 55.55555555555556
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 2 0 4]]
kNN %): 44.44444444444444


In [72]:
#TASK 3
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)
gaussianClassifier = GaussianNB()
gaussianClassifier.fit(X_train, y_train)
y_pred = gaussianClassifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("kNN %):", metrics.accuracy_score(y_test, y_pred)*100)

[[4 0 0 0]
 [0 7 0 0]
 [0 0 2 0]
 [1 0 0 4]]
kNN %): 94.44444444444444


In [79]:
#TASK 4
#print(X_train)
#print(y_train)

#distribution for each class' (fruit) features 
classes = []
count = []
means = []
variances = []
results = []

def findWhichGroup(name, itemIndex):
    #for the existing number of classes
    for c in range(len(classes)):
        #if the class exists already
        if(classes[c] == name):
            #increment the count of that class
            count[c] += 1;
            #add its values to the means and variances lists
            for i in range(len(X_train[itemIndex])):
                means[c][i] += (X_train[itemIndex][i])
                variances[c][i] += (X_train[itemIndex][i])**2 
            return
    #if the class doesn't exist:
    #add it to existing class
    classes.append(name)
    #add a count for that class in counting list
    count.append(1)
    #add a list of means and variances for that class to respective lists
    means.append(X_train[itemIndex].copy())
    variances.append(X_train[itemIndex].copy())
    for v in range(len(variances[0])):
        variances[len(variances)-1][v] = variances[len(variances)-1][v] ** 2
    
    
def calculateParams():
    #calculate means
    for c in range(len(means)):
        for i in range(len(means[c])):
            means[c][i] = means[c][i] / count[c]
    #calculate variances
    for c in range(len(variances)):
        for i in range(len(variances[c])):
            variances[c][i] = (variances[c][i] / count[c]) - (means[c][i]**2)
            
def calculateProbability(value, mean, variance):
    #function for normal distribution
    return (1/(math.sqrt(2*math.pi*variance)))*exp(-(((value-mean)**2)/2*variance))
    
def decideProbability(testItem):
    #store current 'guess' at what item is and at what likelihood
    curResult = ''
    curProb = 0.0
    newProb = 1.0
    for testClass in range(len(classes)):
        #for every class available
        print(classes[testClass],' ', newProb)
        for testFeature in range(len(testItem)):
            #for every feature of that class, calculate probability it is of that class
            newProb *= calculateProbability(testItem[testFeature],means[testClass][testFeature],variances[testClass][testFeature])
            print(newProb)
        #if item more likely to be of new class than current guess class
        if(newProb > curProb):
            #switch guess to new class, likelihood = new likelihood
            curResult = classes[testClass]
            curProb = newProb
        #resut the new likelihood for each class
        newProb = 1
    #return the guess for this item
    return curResult

def testModel():
    for testItem in range(len(X_test)):
        results.append(decideProbability(X_test[testItem]))
    
def calculateAccuracy():
    count = 0
    for resultItem in range(len(results)):
        if(results[resultItem] == y_test[resultItem]):
            count += 1
    print("Accuracy: ",count/len(results)*100)

items = len(X_train)
for x in range(0,items):
    if(y_train[x] == 'orange'): { findWhichGroup('orange',x)}
    elif(y_train[x] == 'lemon'): { findWhichGroup('lemon',x)}
    elif(y_train[x] == 'apple'): { findWhichGroup('apple',x)}
    elif(y_train[x] == 'mandarin'): { findWhichGroup('mandarin',x)}

calculateParams()
testModel()

print("y_test: ", y_test)
print("my results: ",results)
calculateAccuracy()

orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
0.0
0.0
0.0
0.0
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
0.0
0.0
0.0
0.0
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
1.0849680897827136e-124
1.215750103150457e-124
1.6731345776784402e-124
5.1290192185630665e-124
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
0.0
0.0
0.0
0.0
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
0.0
0.0
0.0
0.0
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
1.0849680897827136e-124
1.2195654276939467e-124
1.6761035897040056e-124
5.137842315656119e-124
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
0.0
0.0
0.0
apple   1
0.0
0.0
0.0
0.0
orange   1.0
0.0
0.0
0.0
0.0
mandarin   1
0.0
0.0
0.0
0.0
lemon   1
0.0
